<a href="https://colab.research.google.com/github/radf0001/NLP/blob/main/Pra%CC%81ctica_1_Extraccio%CC%81n_de_Informacio%CC%81n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 1 ( Extracción de Información)

## Profesora: Lisibonny Beato
### Período 3-2022-2023</div>


In [ ]:
!python -m pip install textacy
!python -m pip install visualise_spacy_tree
!python -m spacy download en_core_web_trf
!python -m pip install spacy_transformers
!python  -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: pydot
    Found existing installation: pydot 1.4.2
    Uninstalling pydot-1.4.2:
      Successfully uninstalled pydot-1.4.2
2023-06-28 01:17:06.857194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 01:17:07.861656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━

In [ ]:
# Importando algunas librerias

import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher
import os
import sys
import en_core_web_trf
import spacy_transformers
import re
import nltk
import string
from nltk.corpus import stopwords

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>Para esta práctica estarán utilizando el corpus WNUT16, el cual encontrarán en la carpeta de dataset de nuestro curso. Para más información sobre este corpus ver el siguiente paper: https://aclanthology.org/W16-3919.pdf
<br />
<br />
Aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Nivel Básico
### Puntuación máxima de la tarea: 5 puntos
#### Limpieza y preparación de los datos, Extracción de palabras clave (KPE),  y reconocimiento de entidades relevantes (NER) con modelos heurísticos y pre-entrenados para  reconocer al menos 3 tipos de entidades.

In [ ]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

In [ ]:
# Cargando Dataset
df = pd.DataFrame(columns=["Oracion"])
i = 0
oracion = ""
with open("/content/wnut16_test.txt") as bookfile:
    for line in bookfile:
        fields = line.split("\t")
        if(fields[0] != "\n"):
          if("http" not in fields[0]):
            oracion += fields[0] + " "
        else:
          df.loc[i, "Oracion"] = oracion;
          oracion = ""
          i += 1

In [ ]:
df

,Oracion
0,New Orleans Mother 's Day Parade shooting . One of the people hurt was a 10-year-old girl . WHAT THE HELL IS WRONG WITH PEOPLE ?
1,RT @hxranspizza : Going into school tomorrow like #KCA #Vote1DUK
2,May e just a smile in your heart EILY CountdownBegins #PushAwardsLizQuens
3,I could so do Thursday Club right now
4,@therealdaftbear Albert Nobbs ( Glenn Close)is a woman living as a man in order to find work in the harsh environment of 19th-century Ireland
...,...
3845,"Priest killed , another injured in US shooting : LOS ANGELES -- Police were hunting Thursday a gunman or gunmen who ..."
3846,"Michael__Myerz : |LIVE NOW| Yes #meerkat January 03 , 2016 at 06:54 AM"
3847,Daily Fantasy Basketball 2015 : Best NBA DFS DraftKings Lineup for March 22 #SportsTap
3848,@Toniakins no man alive has it all . But you can live enough in d moment knowing nothing in life is promised except tomorrow . Enjoy today


In [ ]:
def limpiar(text):
    # Quitando algunos caracteres
    text = re.sub('\n', ' ', str(text))
    text = re.sub('\n ', ' ', str(text))
    text = re.sub('-', ' ', str(text))
    text = re.sub('- ', ' ', str(text))
    text = re.sub('"', ' ', str(text))
    text = re.sub("'s", ' ', str(text))
    text = re.sub(r"[^a-zA-Z0-9]", ' ', str(text))

    return text

def longitud(text):
  return len(text.split())

In [ ]:
df['Oracion_Limpia']=df['Oracion'].apply(limpiar)
df['Oracion_Limpia']=df['Oracion_Limpia'].apply(lambda x: ' '.join(x.split()))
df['Longitud']=df['Oracion_Limpia'].apply(longitud)


In [ ]:
df

### Extraccion de KPES

In [ ]:
# Vamos a tomar solo las primeras 4000 oraciones, ya que la funcion que convierte a documento de spacy no permite
# mas de 1000000 de caracteres para el documento.
oraciones_sample=df['Oracion_Limpia'][0:4000]

In [ ]:
all_words = ' '.join([word for word in oraciones_sample])
all_words=all_words.lower()
all_words

'new orleans mother day parade shooting one of the people hurt was a 10 year old girl what the hell is wrong with people rt hxranspizza going into school tomorrow like kca vote1duk may e just a smile in your heart eily countdownbegins pushawardslizquens i could so do thursday club right now therealdaftbear albert nobbs glenn close is a woman living as a man in order to find work in the harsh environment of 19th century ireland rt 1dasiacrew louis interview with the sun about his own record label 78 productions limited proudoflouis 9 injured in minneapolis nightclub shooting vikings player among victims so happy moving into my new place on tuesday rt aldenrichards02 happy 10th weeksary sa ating lahat aldubourmissionislove ikawyung tangi kung hiling sa 18th bday ko i drive by that motel almost every night mesashooting apple macbook pro a1278 13 3 laptop md101ll a june 2012 full read by ebay tuff culture destiny ep par 042 forthcoming 27th july via juno tracklist destiny questions theres 

In [ ]:
# Voy a convertir el string con todos los speeches a un formato de documento spacy
speeches=textacy.make_spacy_doc(all_words, lang='en_core_web_sm')

In [ ]:
# Con este linea pedimos que se obtengan los mejores N KPEs desde el texto
textacy.extract.keyterms.textrank(speeches,topn=10)

[('free trial rt diss julia winnipegwatch rt ailysconfront rt dulayrdulay23 tuesday afternoon winnipeg',
  0.07158372968148383),
 ('rt skizzymar tomorrow rt grimk c une pute mon freeeeeee audreyann68',
  0.05333022591800981),
 ('monday rt pojoktweet silakan rt manual tweet ini amp follow pojoktweet ada pulsa 10rb utk',
  0.05259090142174116),
 ('alexsalmond rt triesennora sunday sunday martoncsokas rt avgsociaimedia twitter happy 4th',
  0.05249897286658552),
 ('rt ithomaspenaflor rt sa mga kinikilig jan wala pa man aldubmeetstvj snapchat',
  0.05147209240487351),
 ('ka ngayon meron din ako yung pagmumura mo ang isa dun rt hornyskydragon rt',
  0.05034004210141763),
 ('excited rikerr5 rt storm ntlegs11 friday night nath werunasone purplepride rt ms sadel',
  0.04955316877100509),
 ('justinformmva rt cavs mania c mon cavs rt bhr hospital',
  0.049518432248980046),
 ('marysville washington ne rt ofcprllvrs mana nih prillvers yang belom beli tiket anniv ke 4th rt dunyanews nab',
  0.04859

### Encontrando NERs con modelo preentrenado.

In [ ]:
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_trf')
#nlp = spacy.load('en_core_web_lg')

In [ ]:
entidades=nlp(all_words).ents
entidades

In [ ]:
{(ent.text.strip(), ent.label_) for ent in nlp(all_words).ents}

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio
### Puntuación máxima de la tarea: 2 puntos
#### Entrenar un modelo propio de NER mediante el pipeline de Spacy

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 3 puntos
#### Entrenar un modelo de NER mediante el uso de Conditional Random Fields. Recuerde ir más allá que donde llega el ejemplo al respecto provisto en clases.